GET PLAYLIST INFORMATION

In [ ]:
import json
import threading
import time
from seleniumwire import webdriver
import gzip
import brotli

# === CONFIG ===
url_to_open = "https://open.spotify.com/playlist/797sNaO2hxZrd43peAsW3G?si=10c5de9fe4654f1c"
output_file = "spotify_pathfinder_responses.json"
target_api_url = "https://api-partner.spotify.com/pathfinder/v2/query"

# === GLOBALS ===
captured_data = []
seen_requests = set()
stop_capture = False

# === FUNCTION: Decode response body ===
def decode_response_body(response):
    """Decode response body handling different compression formats"""
    try:
        body = response.body
        if not body:
            return ""
        
        # Check content encoding and decode accordingly
        encoding = response.headers.get('content-encoding', '').lower()
        
        if encoding == 'gzip':
            body = gzip.decompress(body)
        elif encoding == 'br':
            body = brotli.decompress(body)
        elif encoding == 'deflate':
            import zlib
            body = zlib.decompress(body)
        
        # Try to decode as UTF-8
        try:
            return body.decode('utf-8')
        except UnicodeDecodeError:
            return body.decode('utf-8', errors='ignore')
    except Exception as e:
        print(f"[!] Error decoding response body: {e}")
        return ""

# === FUNCTION: Parse JSON response ===
def parse_json_response(body_text):
    """Try to parse response as JSON"""
    try:
        return json.loads(body_text)
    except json.JSONDecodeError:
        return body_text

# === FUNCTION: Listen for "stop" command ===
def listen_for_stop():
    global stop_capture
    while True:
        user_input = input("Type 'stop' to end capturing and save the file:\n>>> ")
        if user_input.strip().lower() == "stop":
            stop_capture = True
            break

# === FUNCTION: Start capturing requests ===
def capture_requests():
    global stop_capture
    pathfinder_count = 0
    
    while not stop_capture:
        for request in driver.requests:
            # Only capture Pathfinder API requests
            if (request.response and 
                request.id not in seen_requests and 
                target_api_url in request.url):
                
                seen_requests.add(request.id)
                pathfinder_count += 1
                
                print(f"🎯 Captured Pathfinder API request #{pathfinder_count}")
                print(f"   URL: {request.url}")
                print(f"   Method: {request.method}")
                print(f"   Status: {request.response.status_code}")
                
                try:
                    # Decode request body
                    request_body = ""
                    if request.body:
                        try:
                            request_body = request.body.decode('utf-8')
                        except UnicodeDecodeError:
                            request_body = request.body.decode('utf-8', errors='ignore')
                    
                    # Decode response body
                    response_body = decode_response_body(request.response)
                    
                    # Try to parse response as JSON for better formatting
                    parsed_response = parse_json_response(response_body)
                    
                    # Capture comprehensive data
                    captured_data.append({
                        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
                        "request_id": request.id,
                        "url": request.url,
                        "method": request.method,
                        "request_headers": dict(request.headers),
                        "request_body": request_body,
                        "request_body_parsed": parse_json_response(request_body) if request_body else None,
                        "response": {
                            "status_code": request.response.status_code,
                            "status_reason": request.response.reason,
                            "headers": dict(request.response.headers),
                            "body_raw": response_body,
                            "body_parsed": parsed_response,
                            "body_length": len(response_body) if response_body else 0
                        },
                        "query_params": dict(request.params) if hasattr(request, 'params') else {},
                    })
                    
                    print(f"   Response length: {len(response_body)} characters")
                    if isinstance(parsed_response, dict):
                        print(f"   JSON keys: {list(parsed_response.keys()) if parsed_response else 'None'}")
                    print("   ✅ Captured successfully\n")
                    
                except Exception as e:
                    print(f"[!] Error capturing Pathfinder request: {e}")
                    # Still try to capture basic info
                    captured_data.append({
                        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
                        "request_id": request.id,
                        "url": request.url,
                        "method": request.method,
                        "error": str(e),
                        "response_status": request.response.status_code if request.response else None
                    })
        
        time.sleep(1)  # More frequent checking for API calls

# === SETUP SELENIUM DRIVER ===
print("🔄 Launching browser...")
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
# Disable some security features that might interfere with request capture
options.add_argument("--disable-web-security")
options.add_argument("--allow-running-insecure-content")

driver = webdriver.Chrome(options=options)

# Clear any existing requests
driver.requests.clear()

driver.get(url_to_open)
print(f"🌐 Opened {url_to_open}")
print(f"🎯 Monitoring for requests to: {target_api_url}")
print("🟢 You can now interact with the page (e.g. play songs, scroll, search).")
print("   The script will automatically capture Pathfinder API calls.")

# === START THREADS ===
stop_thread = threading.Thread(target=listen_for_stop)
stop_thread.daemon = True
stop_thread.start()

capture_requests()

# === SAVE OUTPUT ===
print("🛑 Stopping capture. Writing data to file...")

# Create a more structured output
output_data = {
    "capture_info": {
        "target_url": url_to_open,
        "target_api": target_api_url,
        "capture_time": time.strftime("%Y-%m-%d %H:%M:%S"),
        "total_requests_captured": len(captured_data)
    },
    "requests": captured_data
}

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"✅ Captured {len(captured_data)} Pathfinder API requests saved to '{output_file}'")

# Print summary
if captured_data:
    print("\n📊 CAPTURE SUMMARY:")
    for i, req in enumerate(captured_data, 1):
        print(f"   {i}. {req['method']} - Status: {req.get('response', {}).get('status_code', 'N/A')}")
        if 'response' in req and 'body_parsed' in req['response']:
            body = req['response']['body_parsed']
            if isinstance(body, dict):
                print(f"      Keys: {', '.join(list(body.keys())[:5])}{'...' if len(body.keys()) > 5 else ''}")

# === CLEAN UP ===
driver.quit()
print("✅ Browser closed.")

🔄 Launching browser...
🌐 Opened https://open.spotify.com/playlist/797sNaO2hxZrd43peAsW3G?si=10c5de9fe4654f1c
🎯 Monitoring for requests to: https://api-partner.spotify.com/pathfinder/v2/query
🟢 You can now interact with the page (e.g. play songs, scroll, search).
   The script will automatically capture Pathfinder API calls.
🎯 Captured Pathfinder API request #1
   URL: https://api-partner.spotify.com/pathfinder/v2/query
   Method: POST
   Status: 200
   Response length: 45911 characters
   JSON keys: ['data', 'extensions']
   ✅ Captured successfully

🎯 Captured Pathfinder API request #2
   URL: https://api-partner.spotify.com/pathfinder/v2/query
   Method: POST
   Status: 200
   Response length: 651 characters
   JSON keys: ['data', 'extensions']
   ✅ Captured successfully

🎯 Captured Pathfinder API request #3
   URL: https://api-partner.spotify.com/pathfinder/v2/query
   Method: POST
   Status: 200
   Response length: 145 characters
   JSON keys: ['data', 'extensions']
   ✅ Captured su

GET THE PARTICLAR SONGS API'S ALONE

In [8]:
import json
import threading
import time
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import gzip
import brotli

# === CONFIG ===
url_to_open = "https://open.spotify.com/playlist/797sNaO2hxZrd43peAsW3G?si=10c5de9fe4654f1c"
output_file = "spotify_playlist_items.json"
target_api_url = "https://api-partner.spotify.com/pathfinder/v2/query"

# Scroll configuration
SCROLL_PAUSE_TIME = 2  # Seconds to wait between scrolls
AUTO_SCROLL_ENABLED = True  # Set to False to disable auto-scrolling
SCROLL_PIXELS = 800  # Number of pixels to scroll each time

# === GLOBALS ===
captured_data = []
seen_requests = set()
stop_capture = False
auto_scroll_active = False

# === FUNCTION: Decode response body ===
def decode_response_body(response):
    """Decode response body handling different compression formats"""
    try:
        body = response.body
        if not body:
            return ""
        
        # Check content encoding and decode accordingly
        encoding = response.headers.get('content-encoding', '').lower()
        
        if encoding == 'gzip':
            body = gzip.decompress(body)
        elif encoding == 'br':
            body = brotli.decompress(body)
        elif encoding == 'deflate':
            import zlib
            body = zlib.decompress(body)
        
        # Try to decode as UTF-8
        try:
            return body.decode('utf-8')
        except UnicodeDecodeError:
            return body.decode('utf-8', errors='ignore')
    except Exception as e:
        print(f"[!] Error decoding response body: {e}")
        return ""

# === FUNCTION: Parse JSON response ===
def parse_json_response(body_text):
    """Try to parse response as JSON"""
    try:
        return json.loads(body_text)
    except json.JSONDecodeError:
        return body_text

# === FUNCTION: Check if response contains PlaylistItemsPage ===
def is_playlist_items_response(parsed_response):
    """Check if the response contains playlist items data"""
    try:
        if isinstance(parsed_response, dict):
            data = parsed_response.get('data', {})
            playlist_v2 = data.get('playlistV2', {})
            content = playlist_v2.get('content', {})
            return content.get('__typename') == 'PlaylistItemsPage'
        return False
    except:
        return False

# === FUNCTION: Extract pagination info ===
def extract_pagination_info(parsed_response):
    """Extract pagination information from the response"""
    try:
        if isinstance(parsed_response, dict):
            data = parsed_response.get('data', {})
            playlist_v2 = data.get('playlistV2', {})
            content = playlist_v2.get('content', {})
            paging_info = content.get('pagingInfo', {})
            items = content.get('items', [])
            
            return {
                'limit': paging_info.get('limit', 0),
                'offset': paging_info.get('offset', 0),
                'totalCount': paging_info.get('totalCount', 0),
                'items_in_response': len(items)
            }
    except:
        pass
    return None

# === FUNCTION: Auto-scroll the page ===
def auto_scroll():
    global stop_capture, auto_scroll_active
    auto_scroll_active = True
    scroll_count = 0
    
    print("🔄 Starting auto-scroll...")
    
    try:
        # Wait for page to load initially
        time.sleep(3)
        
        while not stop_capture and AUTO_SCROLL_ENABLED:
            try:
                # Get current scroll position
                current_scroll = driver.execute_script("return window.pageYOffset;")
                page_height = driver.execute_script("return document.body.scrollHeight;")
                window_height = driver.execute_script("return window.innerHeight;")
                
                # Scroll down
                driver.execute_script(f"window.scrollBy(0, {SCROLL_PIXELS});")
                scroll_count += 1
                
                print(f"🔽 Scroll #{scroll_count} - Position: {current_scroll}px")
                
                # Wait for content to load
                time.sleep(SCROLL_PAUSE_TIME)
                
                # Check if we've reached the bottom
                new_scroll = driver.execute_script("return window.pageYOffset;")
                if new_scroll == current_scroll or new_scroll + window_height >= page_height:
                    print("📍 Reached bottom of page, continuing to monitor...")
                    # Continue monitoring but don't scroll further
                    time.sleep(SCROLL_PAUSE_TIME * 2)
                
            except Exception as e:
                print(f"[!] Error during scrolling: {e}")
                time.sleep(SCROLL_PAUSE_TIME)
                
    except Exception as e:
        print(f"[!] Error in auto-scroll thread: {e}")
    
    auto_scroll_active = False

# === FUNCTION: Listen for commands ===
def listen_for_commands():
    global stop_capture, AUTO_SCROLL_ENABLED
    while True:
        print("\nCommands:")
        print("  'stop' - Stop capturing and save file")
        print("  'scroll on' - Enable auto-scrolling")
        print("  'scroll off' - Disable auto-scrolling")
        print("  'status' - Show current status")
        
        user_input = input(">>> ").strip().lower()
        
        if user_input == "stop":
            stop_capture = True
            break
        elif user_input == "scroll on":
            AUTO_SCROLL_ENABLED = True
            print("✅ Auto-scrolling enabled")
        elif user_input == "scroll off":
            AUTO_SCROLL_ENABLED = False
            print("🛑 Auto-scrolling disabled")
        elif user_input == "status":
            print(f"📊 Status:")
            print(f"   Playlist items captured: {len(captured_data)}")
            print(f"   Auto-scroll: {'ON' if AUTO_SCROLL_ENABLED else 'OFF'}")
            print(f"   Auto-scroll active: {'YES' if auto_scroll_active else 'NO'}")

# === FUNCTION: Start capturing requests ===
def capture_requests():
    global stop_capture
    playlist_items_count = 0
    
    while not stop_capture:
        for request in driver.requests:
            # Only capture Pathfinder API requests
            if (request.response and 
                request.id not in seen_requests and 
                target_api_url in request.url):
                
                seen_requests.add(request.id)
                
                try:
                    # Decode response body
                    response_body = decode_response_body(request.response)
                    parsed_response = parse_json_response(response_body)
                    
                    # Check if this is a playlist items response
                    if is_playlist_items_response(parsed_response):
                        playlist_items_count += 1
                        pagination_info = extract_pagination_info(parsed_response)
                        
                        print(f"🎯 Captured Playlist Items Request #{playlist_items_count}")
                        print(f"   URL: {request.url}")
                        print(f"   Status: {request.response.status_code}")
                        
                        if pagination_info:
                            print(f"   📄 Pagination: Offset {pagination_info['offset']}, "
                                  f"Limit {pagination_info['limit']}, "
                                  f"Items: {pagination_info['items_in_response']}, "
                                  f"Total: {pagination_info['totalCount']}")
                        
                        # Decode request body
                        request_body = ""
                        if request.body:
                            try:
                                request_body = request.body.decode('utf-8')
                            except UnicodeDecodeError:
                                request_body = request.body.decode('utf-8', errors='ignore')
                        
                        # Capture comprehensive data
                        captured_data.append({
                            "capture_sequence": playlist_items_count,
                            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
                            "request_id": request.id,
                            "url": request.url,
                            "method": request.method,
                            "request_headers": dict(request.headers),
                            "request_body": request_body,
                            "request_body_parsed": parse_json_response(request_body) if request_body else None,
                            "response": {
                                "status_code": request.response.status_code,
                                "status_reason": request.response.reason,
                                "headers": dict(request.response.headers),
                                "body_raw": response_body,
                                "body_parsed": parsed_response,
                                "body_length": len(response_body) if response_body else 0
                            },
                            "pagination_info": pagination_info,
                            "query_params": dict(request.params) if hasattr(request, 'params') else {},
                        })
                        
                        print(f"   Response length: {len(response_body)} characters")
                        print("   ✅ Playlist items captured successfully\n")
                    
                    # Still track non-playlist requests for debugging (but don't save them)
                    else:
                        print(f"⏭️  Skipped non-playlist request to {request.url}")
                        
                except Exception as e:
                    print(f"[!] Error processing Pathfinder request: {e}")
        
        time.sleep(0.5)  # Frequent checking for API calls

# === SETUP SELENIUM DRIVER ===
print("🔄 Launching browser...")
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
# Disable some security features that might interfere with request capture
options.add_argument("--disable-web-security")
options.add_argument("--allow-running-insecure-content")
# Add user agent to look more like a real browser
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)

# Clear any existing requests
driver.requests.clear()

driver.get(url_to_open)
print(f"🌐 Opened {url_to_open}")
print(f"🎯 Monitoring for PlaylistItemsPage requests to: {target_api_url}")
print("🟢 The script will automatically scroll and capture playlist items.")

# === START THREADS ===
command_thread = threading.Thread(target=listen_for_commands)
command_thread.daemon = True
command_thread.start()

if AUTO_SCROLL_ENABLED:
    scroll_thread = threading.Thread(target=auto_scroll)
    scroll_thread.daemon = True
    scroll_thread.start()

capture_requests()

# === SAVE OUTPUT ===
print("🛑 Stopping capture. Writing data to file...")

# Create a more structured output
output_data = {
    "capture_info": {
        "target_url": url_to_open,
        "target_api": target_api_url,
        "capture_time": time.strftime("%Y-%m-%d %H:%M:%S"),
        "total_playlist_items_requests": len(captured_data),
        "filter_criteria": "Only PlaylistItemsPage responses captured"
    },
    "playlist_items_requests": captured_data
}

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"✅ Captured {len(captured_data)} Playlist Items requests saved to '{output_file}'")

# Print summary
if captured_data:
    print("\n📊 CAPTURE SUMMARY:")
    for i, req in enumerate(captured_data, 1):
        paging = req.get('pagination_info', {})
        print(f"   {i}. Offset: {paging.get('offset', 'N/A')}, "
              f"Items: {paging.get('items_in_response', 'N/A')}, "
              f"Total: {paging.get('totalCount', 'N/A')}")

# === CLEAN UP ===
driver.quit()
print("✅ Browser closed.")

🔄 Launching browser...
🌐 Opened https://open.spotify.com/playlist/797sNaO2hxZrd43peAsW3G?si=10c5de9fe4654f1c
🎯 Monitoring for PlaylistItemsPage requests to: https://api-partner.spotify.com/pathfinder/v2/query
🟢 The script will automatically scroll and capture playlist items.

Commands:
  'stop' - Stop capturing and save file
  'scroll on' - Enable auto-scrolling
  'scroll off' - Disable auto-scrolling
  'status' - Show current status
🔄 Starting auto-scroll...
🔽 Scroll #1 - Position: 0px
🎯 Captured Playlist Items Request #1
   URL: https://api-partner.spotify.com/pathfinder/v2/query
   Status: 200
   📄 Pagination: Offset 0, Limit 25, Items: 25, Total: 0
   Response length: 45911 characters
   ✅ Playlist items captured successfully

⏭️  Skipped non-playlist request to https://api-partner.spotify.com/pathfinder/v2/query
⏭️  Skipped non-playlist request to https://api-partner.spotify.com/pathfinder/v2/query
⏭️  Skipped non-playlist request to https://api-partner.spotify.com/pathfinder/v2/q

GETTING ITEMS IN SINGLE FILE

In [10]:
import json
import threading
import time
import os
from datetime import datetime
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import gzip
import brotli

# === CONFIG ===
url_to_open = "https://open.spotify.com/playlist/797sNaO2hxZrd43peAsW3G?si=10c5de9fe4654f1c"
target_api_url = "https://api-partner.spotify.com/pathfinder/v2/query"

# Create timestamped folder
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_folder = f"spotify_capture_{timestamp}"
os.makedirs(output_folder, exist_ok=True)

output_file = os.path.join(output_folder, "spotify_playlist_requests.json")
items_file = os.path.join(output_folder, "all_playlist_items.json")

# Scroll configuration
SCROLL_PAUSE_TIME = 2  # Seconds to wait between scrolls
AUTO_SCROLL_ENABLED = True  # Set to False to disable auto-scrolling
SCROLL_PIXELS = 800  # Number of pixels to scroll each time

# === GLOBALS ===
captured_data = []
all_playlist_items = []  # Store all items from all requests
seen_requests = set()
stop_capture = False
auto_scroll_active = False

# === FUNCTION: Decode response body ===
def decode_response_body(response):
    """Decode response body handling different compression formats"""
    try:
        body = response.body
        if not body:
            return ""
        
        # Check content encoding and decode accordingly
        encoding = response.headers.get('content-encoding', '').lower()
        
        if encoding == 'gzip':
            body = gzip.decompress(body)
        elif encoding == 'br':
            body = brotli.decompress(body)
        elif encoding == 'deflate':
            import zlib
            body = zlib.decompress(body)
        
        # Try to decode as UTF-8
        try:
            return body.decode('utf-8')
        except UnicodeDecodeError:
            return body.decode('utf-8', errors='ignore')
    except Exception as e:
        print(f"[!] Error decoding response body: {e}")
        return ""

# === FUNCTION: Parse JSON response ===
def parse_json_response(body_text):
    """Try to parse response as JSON"""
    try:
        return json.loads(body_text)
    except json.JSONDecodeError:
        return body_text

# === FUNCTION: Check if response contains PlaylistItemsPage ===
def is_playlist_items_response(parsed_response):
    """Check if the response contains playlist items data"""
    try:
        if isinstance(parsed_response, dict):
            data = parsed_response.get('data', {})
            playlist_v2 = data.get('playlistV2', {})
            content = playlist_v2.get('content', {})
            return content.get('__typename') == 'PlaylistItemsPage'
        return False
    except:
        return False

# === FUNCTION: Extract pagination info and items ===
def extract_pagination_info(parsed_response):
    """Extract pagination information from the response"""
    try:
        if isinstance(parsed_response, dict):
            data = parsed_response.get('data', {})
            playlist_v2 = data.get('playlistV2', {})
            content = playlist_v2.get('content', {})
            paging_info = content.get('pagingInfo', {})
            items = content.get('items', [])
            
            return {
                'limit': paging_info.get('limit', 0),
                'offset': paging_info.get('offset', 0),
                'totalCount': paging_info.get('totalCount', 0),
                'items_in_response': len(items)
            }
    except:
        pass
    return None

# === FUNCTION: Extract items from response ===
def extract_items_from_response(parsed_response):
    """Extract the items array from playlist response"""
    try:
        if isinstance(parsed_response, dict):
            data = parsed_response.get('data', {})
            playlist_v2 = data.get('playlistV2', {})
            content = playlist_v2.get('content', {})
            items = content.get('items', [])
            return items
    except:
        pass
    return []

# === FUNCTION: Auto-scroll the page ===
def auto_scroll():
    global stop_capture, auto_scroll_active
    auto_scroll_active = True
    scroll_count = 0
    
    print("🔄 Starting auto-scroll...")
    
    try:
        # Wait for page to load initially
        time.sleep(3)
        
        while not stop_capture and AUTO_SCROLL_ENABLED:
            try:
                # Get current scroll position
                current_scroll = driver.execute_script("return window.pageYOffset;")
                page_height = driver.execute_script("return document.body.scrollHeight;")
                window_height = driver.execute_script("return window.innerHeight;")
                
                # Scroll down
                driver.execute_script(f"window.scrollBy(0, {SCROLL_PIXELS});")
                scroll_count += 1
                
                print(f"🔽 Scroll #{scroll_count} - Position: {current_scroll}px")
                
                # Wait for content to load
                time.sleep(SCROLL_PAUSE_TIME)
                
                # Check if we've reached the bottom
                new_scroll = driver.execute_script("return window.pageYOffset;")
                if new_scroll == current_scroll or new_scroll + window_height >= page_height:
                    print("📍 Reached bottom of page, continuing to monitor...")
                    # Continue monitoring but don't scroll further
                    time.sleep(SCROLL_PAUSE_TIME * 2)
                
            except Exception as e:
                print(f"[!] Error during scrolling: {e}")
                time.sleep(SCROLL_PAUSE_TIME)
                
    except Exception as e:
        print(f"[!] Error in auto-scroll thread: {e}")
    
    auto_scroll_active = False

# === FUNCTION: Listen for commands ===
def listen_for_commands():
    global stop_capture, AUTO_SCROLL_ENABLED
    while True:
        print("\nCommands:")
        print("  'stop' - Stop capturing and save files")
        print("  'scroll on' - Enable auto-scrolling")
        print("  'scroll off' - Disable auto-scrolling")
        print("  'status' - Show current status")
        print("  'items' - Show total items collected")
        
        user_input = input(">>> ").strip().lower()
        
        if user_input == "stop":
            stop_capture = True
            break
        elif user_input == "scroll on":
            AUTO_SCROLL_ENABLED = True
            print("✅ Auto-scrolling enabled")
        elif user_input == "scroll off":
            AUTO_SCROLL_ENABLED = False
            print("🛑 Auto-scrolling disabled")
        elif user_input == "status":
            print(f"📊 Status:")
            print(f"   Playlist requests captured: {len(captured_data)}")
            print(f"   Total items collected: {len(all_playlist_items)}")
            print(f"   Auto-scroll: {'ON' if AUTO_SCROLL_ENABLED else 'OFF'}")
            print(f"   Auto-scroll active: {'YES' if auto_scroll_active else 'NO'}")
            print(f"   Output folder: {output_folder}")
        elif user_input == "items":
            print(f"📚 Total items collected: {len(all_playlist_items)}")
            if all_playlist_items:
                print(f"   Latest item example keys: {list(all_playlist_items[-1].keys()) if all_playlist_items[-1] else 'None'}")

# === FUNCTION: Start capturing requests ===
def capture_requests():
    global stop_capture, all_playlist_items
    playlist_items_count = 0
    
    while not stop_capture:
        for request in driver.requests:
            # Only capture Pathfinder API requests
            if (request.response and 
                request.id not in seen_requests and 
                target_api_url in request.url):
                
                seen_requests.add(request.id)
                
                try:
                    # Decode response body
                    response_body = decode_response_body(request.response)
                    parsed_response = parse_json_response(response_body)
                    
                    # Check if this is a playlist items response
                    if is_playlist_items_response(parsed_response):
                        playlist_items_count += 1
                        pagination_info = extract_pagination_info(parsed_response)
                        
                        # Extract items from this response
                        items_in_response = extract_items_from_response(parsed_response)
                        
                        print(f"🎯 Captured Playlist Items Request #{playlist_items_count}")
                        print(f"   URL: {request.url}")
                        print(f"   Status: {request.response.status_code}")
                        
                        if pagination_info:
                            print(f"   📄 Pagination: Offset {pagination_info['offset']}, "
                                  f"Limit {pagination_info['limit']}, "
                                  f"Items: {pagination_info['items_in_response']}, "
                                  f"Total: {pagination_info['totalCount']}")
                        
                        print(f"   🎵 Items extracted from this response: {len(items_in_response)}")
                        
                        # Add items to our collection
                        if items_in_response:
                            all_playlist_items.extend(items_in_response)
                            print(f"   📚 Total items collected so far: {len(all_playlist_items)}")
                        
                        # Decode request body
                        request_body = ""
                        if request.body:
                            try:
                                request_body = request.body.decode('utf-8')
                            except UnicodeDecodeError:
                                request_body = request.body.decode('utf-8', errors='ignore')
                        
                        # Capture comprehensive data
                        captured_data.append({
                            "capture_sequence": playlist_items_count,
                            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
                            "request_id": request.id,
                            "url": request.url,
                            "method": request.method,
                            "request_headers": dict(request.headers),
                            "request_body": request_body,
                            "request_body_parsed": parse_json_response(request_body) if request_body else None,
                            "response": {
                                "status_code": request.response.status_code,
                                "status_reason": request.response.reason,
                                "headers": dict(request.response.headers),
                                "body_raw": response_body,
                                "body_parsed": parsed_response,
                                "body_length": len(response_body) if response_body else 0
                            },
                            "pagination_info": pagination_info,
                            "items_count_in_response": len(items_in_response),
                            "query_params": dict(request.params) if hasattr(request, 'params') else {},
                        })
                        
                        print(f"   Response length: {len(response_body)} characters")
                        print("   ✅ Playlist items captured successfully\n")
                    
                    # Still track non-playlist requests for debugging (but don't save them)
                    else:
                        print(f"⏭️  Skipped non-playlist request to {request.url}")
                        
                except Exception as e:
                    print(f"[!] Error processing Pathfinder request: {e}")
        
        time.sleep(0.5)  # Frequent checking for API calls

# === SETUP SELENIUM DRIVER ===
print("🔄 Launching browser...")
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
# Disable some security features that might interfere with request capture
options.add_argument("--disable-web-security")
options.add_argument("--allow-running-insecure-content")
# Add user agent to look more like a real browser
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)

# Clear any existing requests
driver.requests.clear()

driver.get(url_to_open)
print(f"🌐 Opened {url_to_open}")
print(f"🎯 Monitoring for PlaylistItemsPage requests to: {target_api_url}")
print(f"📁 Output folder: {output_folder}")
print("🟢 The script will automatically scroll and capture playlist items.")

# === START THREADS ===
command_thread = threading.Thread(target=listen_for_commands)
command_thread.daemon = True
command_thread.start()

if AUTO_SCROLL_ENABLED:
    scroll_thread = threading.Thread(target=auto_scroll)
    scroll_thread.daemon = True
    scroll_thread.start()

capture_requests()

# === SAVE OUTPUT ===
print("🛑 Stopping capture. Writing data to files...")

# Create a more structured output for complete requests
output_data = {
    "capture_info": {
        "target_url": url_to_open,
        "target_api": target_api_url,
        "capture_time": time.strftime("%Y-%m-%d %H:%M:%S"),
        "total_playlist_requests": len(captured_data),
        "total_items_collected": len(all_playlist_items),
        "filter_criteria": "Only PlaylistItemsPage responses captured"
    },
    "playlist_items_requests": captured_data
}

# Save complete requests data
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

# Create items-only output
items_output = {
    "extraction_info": {
        "source_playlist_url": url_to_open,
        "extraction_time": time.strftime("%Y-%m-%d %H:%M:%S"),
        "total_items": len(all_playlist_items),
        "requests_processed": len(captured_data)
    },
    "items": all_playlist_items
}

# Save items-only data
with open(items_file, "w", encoding="utf-8") as f:
    json.dump(items_output, f, indent=2, ensure_ascii=False)

print(f"✅ Captured {len(captured_data)} requests saved to '{output_file}'")
print(f"✅ Extracted {len(all_playlist_items)} items saved to '{items_file}'")
print(f"📁 All files saved in folder: {output_folder}")

# Print summary
if captured_data:
    print("\n📊 CAPTURE SUMMARY:")
    for i, req in enumerate(captured_data, 1):
        paging = req.get('pagination_info', {})
        items_count = req.get('items_count_in_response', 0)
        print(f"   {i}. Offset: {paging.get('offset', 'N/A')}, "
              f"Items captured: {items_count}, "
              f"Total in playlist: {paging.get('totalCount', 'N/A')}")

if all_playlist_items:
    print(f"\n🎵 ITEMS SUMMARY:")
    print(f"   Total unique items collected: {len(all_playlist_items)}")
    if all_playlist_items:
        # Show sample keys from first item
        sample_keys = list(all_playlist_items[0].keys()) if all_playlist_items[0] else []
        print(f"   Sample item structure keys: {sample_keys[:10]}{'...' if len(sample_keys) > 10 else ''}")

# Create a simple text summary file
summary_file = os.path.join(output_folder, "capture_summary.txt")
with open(summary_file, "w", encoding="utf-8") as f:
    f.write(f"Spotify Playlist Capture Summary\n")
    f.write(f"================================\n\n")
    f.write(f"Capture Time: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Playlist URL: {url_to_open}\n")
    f.write(f"Total Requests Captured: {len(captured_data)}\n")
    f.write(f"Total Items Extracted: {len(all_playlist_items)}\n\n")
    f.write(f"Files Created:\n")
    f.write(f"- {os.path.basename(output_file)} (Complete request data)\n")
    f.write(f"- {os.path.basename(items_file)} (Items only)\n")
    f.write(f"- {os.path.basename(summary_file)} (This summary)\n")

print(f"📋 Summary saved to '{summary_file}'")

# === CLEAN UP ===
driver.quit()
print("✅ Browser closed.")

🔄 Launching browser...
🌐 Opened https://open.spotify.com/playlist/797sNaO2hxZrd43peAsW3G?si=10c5de9fe4654f1c
🎯 Monitoring for PlaylistItemsPage requests to: https://api-partner.spotify.com/pathfinder/v2/query
📁 Output folder: spotify_capture_20250722_121116
🟢 The script will automatically scroll and capture playlist items.

Commands:
  'stop' - Stop capturing and save files
  'scroll on' - Enable auto-scrolling
  'scroll off' - Disable auto-scrolling
  'status' - Show current status
  'items' - Show total items collected
🔄 Starting auto-scroll...
🔽 Scroll #1 - Position: 0px
📍 Reached bottom of page, continuing to monitor...
🎯 Captured Playlist Items Request #1
   URL: https://api-partner.spotify.com/pathfinder/v2/query
   Status: 200
   📄 Pagination: Offset 0, Limit 25, Items: 25, Total: 0
   🎵 Items extracted from this response: 25
   📚 Total items collected so far: 25
   Response length: 45911 characters
   ✅ Playlist items captured successfully

⏭️  Skipped non-playlist request to h

GET LIST OF TRACK INFO

In [11]:
import json
import os
from datetime import datetime

def extract_track_info(json_file_path, output_file_path=None):
    """
    Extract track names and artist names from Spotify playlist JSON data
    
    Args:
        json_file_path (str): Path to the all_playlist_items.json file
        output_file_path (str): Optional output file path. If not provided, creates one automatically
    """
    
    # Load the JSON data
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"❌ Error: File '{json_file_path}' not found")
        return
    except json.JSONDecodeError:
        print(f"❌ Error: Invalid JSON in file '{json_file_path}'")
        return
    
    # Extract items from the JSON structure
    items = data.get('items', [])
    if not items:
        print("❌ No items found in the JSON file")
        return
    
    print(f"🎵 Found {len(items)} items to process...")
    
    # Extract track information
    tracks_info = []
    skipped_count = 0
    
    for i, item in enumerate(items, 1):
        try:
            # Navigate through the nested structure
            item_v2 = item.get('itemV2', {})
            
            # Check if it's a track (skip if it's not)
            if item_v2.get('__typename') != 'TrackResponseWrapper':
                skipped_count += 1
                continue
                
            track_data = item_v2.get('data', {})
            
            # Extract track name
            track_name = track_data.get('name', 'Unknown Track')
            
            # Extract artist names
            artists_data = track_data.get('artists', {}).get('items', [])
            artist_names = []
            
            for artist in artists_data:
                artist_name = artist.get('profile', {}).get('name', 'Unknown Artist')
                if artist_name not in artist_names:  # Avoid duplicates
                    artist_names.append(artist_name)
            
            # Create track info
            track_info = {
                'track_name': track_name,
                'artists': artist_names,
                'artists_string': ', '.join(artist_names) if artist_names else 'Unknown Artist'
            }
            
            tracks_info.append(track_info)
            
            # Print progress every 100 items
            if i % 100 == 0:
                print(f"✅ Processed {i}/{len(items)} items...")
                
        except Exception as e:
            print(f"⚠️  Error processing item {i}: {e}")
            skipped_count += 1
            continue
    
    print(f"✅ Successfully extracted {len(tracks_info)} tracks")
    if skipped_count > 0:
        print(f"⏭️  Skipped {skipped_count} non-track items")
    
    # Generate output file path if not provided
    if output_file_path is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file_path = f"extracted_tracks_{timestamp}.txt"
    
    # Write to output file
    try:
        with open(output_file_path, 'w', encoding='utf-8') as f:
            f.write(f"Spotify Playlist - Extracted Tracks\n")
            f.write(f"=====================================\n\n")
            f.write(f"Extraction Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Total Tracks: {len(tracks_info)}\n")
            f.write(f"Source File: {json_file_path}\n\n")
            f.write("=" * 60 + "\n\n")
            
            for i, track in enumerate(tracks_info, 1):
                f.write(f"{i:4d}. {track['track_name']}\n")
                f.write(f"       Artists: {track['artists_string']}\n\n")
        
        print(f"📄 Track list saved to: {output_file_path}")
        
    except Exception as e:
        print(f"❌ Error writing to output file: {e}")
        return
    
    # Also create a JSON version for programmatic use
    json_output_path = output_file_path.replace('.txt', '_data.json')
    try:
        json_data = {
            'extraction_info': {
                'extraction_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'total_tracks': len(tracks_info),
                'source_file': json_file_path,
                'skipped_items': skipped_count
            },
            'tracks': tracks_info
        }
        
        with open(json_output_path, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False)
        
        print(f"📄 JSON data saved to: {json_output_path}")
        
    except Exception as e:
        print(f"⚠️  Warning: Could not save JSON version: {e}")
    
    # Print summary
    print(f"\n📊 EXTRACTION SUMMARY:")
    print(f"   Total items processed: {len(items)}")
    print(f"   Tracks extracted: {len(tracks_info)}")
    print(f"   Items skipped: {skipped_count}")
    
    if tracks_info:
        print(f"\n🎵 SAMPLE TRACKS:")
        for i, track in enumerate(tracks_info[:5], 1):  # Show first 5 tracks
            print(f"   {i}. \"{track['track_name']}\" by {track['artists_string']}")
        if len(tracks_info) > 5:
            print(f"   ... and {len(tracks_info) - 5} more tracks")
    
    return tracks_info


# === USAGE EXAMPLES ===
if __name__ == "__main__":
    print("🎵 Spotify Track Extractor")
    print("=" * 40)
    
    # Example usage - you'll need to update these paths
    json_file = input("Enter the path to your all_playlist_items.json file: ").strip()
    
    if not json_file:
        print("Using default filename pattern...")
        # Look for files in current directory
        import glob
        json_files = glob.glob("spotify_capture_*/all_playlist_items.json")
        
        if json_files:
            json_file = json_files[-1]  # Use the most recent one
            print(f"Found: {json_file}")
        else:
            print("❌ No playlist files found. Please provide the full path.")
            exit(1)
    
    # Extract tracks
    result = extract_track_info(json_file)
    
    if result:
        print(f"\n✅ Extraction completed successfully!")
        print(f"🎵 Total tracks extracted: {len(result)}")
    else:
        print("❌ Extraction failed!")

🎵 Spotify Track Extractor
🎵 Found 276 items to process...
✅ Processed 100/276 items...
✅ Processed 200/276 items...
✅ Successfully extracted 276 tracks
📄 Track list saved to: extracted_tracks_20250722_121742.txt
📄 JSON data saved to: extracted_tracks_20250722_121742_data.json

📊 EXTRACTION SUMMARY:
   Total items processed: 276
   Tracks extracted: 276
   Items skipped: 0

🎵 SAMPLE TRACKS:
   1. "Nee Kavithaigala" by Dhibu Ninan Thomas, Pradeep Kumar
   2. "Bodhai Kaname - From "Oh Manapenne"" by Vishal Chandrashekhar, Anirudh Ravichander, Shashaa Tirupati
   3. "Nee Paartha Vizhigal - The Touch of Love" by Anirudh Ravichander, Vijay Yesudas, Shweta Mohan
   4. "Naan Pizhai (From "Kaathuvaakula Rendu Kaadhal")" by Anirudh Ravichander, Ravi G, Shashaa Tirupati
   5. "Oru Devathai (From "Vaamanan")" by Yuvan Shankar Raja, Roop Kumar Rathod
   ... and 271 more tracks

✅ Extraction completed successfully!
🎵 Total tracks extracted: 276


DOWNLOAD SONGS

In [12]:
import json
import os
import re
import time
import subprocess
import sys
from datetime import datetime

# Required installations:
# pip install yt-dlp
# Also need ffmpeg installed on your system

def install_required_packages():
    """Check and install required packages"""
    try:
        import yt_dlp
    except ImportError:
        print("Installing yt-dlp...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "yt-dlp"])
        import yt_dlp
    
    return True

def sanitize_filename(filename):
    """Remove invalid characters from filename"""
    # Remove or replace invalid characters
    filename = re.sub(r'[<>:"/\\|?*]', '', filename)
    filename = re.sub(r'[^\w\s-]', '', filename)
    filename = re.sub(r'[-\s]+', '-', filename)
    return filename.strip('-')[:100]  # Limit length

def search_and_download_audio(track_name, artists, output_folder, max_retries=3):
    """
    Search for and download audio from YouTube
    
    Args:
        track_name (str): Name of the track
        artists (list): List of artist names
        output_folder (str): Folder to save the audio
        max_retries (int): Number of retry attempts
    
    Returns:
        dict: Result information
    """
    
    # Create search query
    artists_str = ' '.join(artists) if isinstance(artists, list) else artists
    search_query = f"{track_name} {artists_str}"
    
    # Sanitize filename
    safe_filename = sanitize_filename(f"{track_name} - {artists_str}")
    output_path = os.path.join(output_folder, f"{safe_filename}.%(ext)s")
    
    # yt-dlp options
    ydl_opts = {
        'format': 'bestaudio/best',
        'extractaudio': True,
        'audioformat': 'mp3',
        'audioquality': '192K',
        'outtmpl': output_path,
        'noplaylist': True,
        'quiet': True,
        'no_warnings': True,
        'default_search': 'ytsearch1:',  # Search YouTube and get first result
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }
    
    result = {
        'track_name': track_name,
        'artists': artists_str,
        'search_query': search_query,
        'status': 'failed',
        'error': None,
        'filename': None,
        'video_title': None,
        'video_url': None
    }
    
    for attempt in range(max_retries):
        try:
            import yt_dlp
            
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                # Search and get info
                search_results = ydl.extract_info(
                    f"ytsearch1:{search_query}",
                    download=False
                )
                
                if not search_results or 'entries' not in search_results or not search_results['entries']:
                    result['error'] = 'No search results found'
                    continue
                
                video_info = search_results['entries'][0]
                result['video_title'] = video_info.get('title', 'Unknown')
                result['video_url'] = video_info.get('webpage_url', '')
                
                # Download the audio
                ydl.download([video_info['webpage_url']])
                
                # Find the downloaded file
                expected_filename = f"{safe_filename}.mp3"
                full_path = os.path.join(output_folder, expected_filename)
                
                if os.path.exists(full_path):
                    result['status'] = 'success'
                    result['filename'] = expected_filename
                    return result
                else:
                    # Look for any file with similar name
                    for file in os.listdir(output_folder):
                        if file.startswith(safe_filename) and file.endswith('.mp3'):
                            result['status'] = 'success'
                            result['filename'] = file
                            return result
                
        except Exception as e:
            result['error'] = str(e)
            if attempt < max_retries - 1:
                print(f"   ⚠️  Attempt {attempt + 1} failed: {e}, retrying...")
                time.sleep(2)  # Wait before retry
            continue
    
    return result

def download_playlist_audio(tracks_json_file, output_base_folder=None, start_from=1, max_downloads=None):
    """
    Download audio for all tracks in the playlist
    
    Args:
        tracks_json_file (str): Path to the extracted tracks JSON file
        output_base_folder (str): Base folder for downloads
        start_from (int): Track number to start from (for resuming)
        max_downloads (int): Maximum number of tracks to download
    """
    
    # Install required packages
    if not install_required_packages():
        print("❌ Failed to install required packages")
        return
    
    # Load tracks data
    try:
        with open(tracks_json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"❌ Error loading tracks file: {e}")
        return
    
    tracks = data.get('tracks', [])
    if not tracks:
        print("❌ No tracks found in the file")
        return
    
    # Create output folder
    if output_base_folder is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_base_folder = f"downloaded_playlist_{timestamp}"
    
    os.makedirs(output_base_folder, exist_ok=True)
    
    # Limit downloads if specified
    if max_downloads:
        tracks = tracks[:max_downloads]
    
    # Skip to starting point if resuming
    if start_from > 1:
        tracks = tracks[start_from-1:]
        print(f"📍 Resuming from track {start_from}")
    
    print(f"🎵 Starting download of {len(tracks)} tracks...")
    print(f"📁 Output folder: {output_base_folder}")
    print(f"⚠️  LEGAL NOTICE: Only download content you have the right to access.")
    print(f"⚠️  Be aware of copyright laws in your jurisdiction.")
    
    # Ask for confirmation
    response = input("\nDo you want to proceed? (y/N): ").strip().lower()
    if response != 'y':
        print("❌ Download cancelled")
        return
    
    # Download tracking
    results = []
    successful_downloads = 0
    failed_downloads = 0
    
    # Create log file
    log_file = os.path.join(output_base_folder, "download_log.txt")
    
    for i, track in enumerate(tracks, start_from):
        print(f"\n🎵 [{i}/{len(tracks) + start_from - 1}] {track['track_name']} - {track['artists_string']}")
        
        try:
            result = search_and_download_audio(
                track['track_name'],
                track['artists'],
                output_base_folder
            )
            
            results.append(result)
            
            if result['status'] == 'success':
                successful_downloads += 1
                print(f"   ✅ Downloaded: {result['filename']}")
                print(f"   🎬 From video: {result['video_title']}")
            else:
                failed_downloads += 1
                print(f"   ❌ Failed: {result['error']}")
            
            # Log result
            with open(log_file, 'a', encoding='utf-8') as f:
                f.write(f"{i}. {track['track_name']} - {track['artists_string']}\n")
                f.write(f"   Status: {result['status']}\n")
                f.write(f"   Video: {result.get('video_title', 'N/A')}\n")
                f.write(f"   Error: {result.get('error', 'None')}\n\n")
            
            # Small delay to be respectful to YouTube
            time.sleep(1)
            
        except KeyboardInterrupt:
            print("\n⏹️  Download interrupted by user")
            break
        except Exception as e:
            print(f"   ❌ Unexpected error: {e}")
            failed_downloads += 1
    
    # Create summary
    summary_file = os.path.join(output_base_folder, "download_summary.json")
    summary_data = {
        'download_info': {
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'source_file': tracks_json_file,
            'output_folder': output_base_folder,
            'total_tracks': len(tracks),
            'successful_downloads': successful_downloads,
            'failed_downloads': failed_downloads,
            'success_rate': f"{(successful_downloads/len(tracks)*100):.1f}%" if tracks else "0%"
        },
        'results': results
    }
    
    with open(summary_file, 'w', encoding='utf-8') as f:
        json.dump(summary_data, f, indent=2, ensure_ascii=False)
    
    # Print final summary
    print(f"\n📊 DOWNLOAD COMPLETE")
    print(f"   Total tracks: {len(tracks)}")
    print(f"   ✅ Successful: {successful_downloads}")
    print(f"   ❌ Failed: {failed_downloads}")
    print(f"   📈 Success rate: {(successful_downloads/len(tracks)*100):.1f}%")
    print(f"   📁 Files saved in: {output_base_folder}")
    print(f"   📋 Detailed log: {log_file}")
    print(f"   📊 Summary: {summary_file}")

def check_prerequisites():
    """Check if required tools are available"""
    print("🔧 Checking prerequisites...")
    
    # Check ffmpeg
    try:
        result = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True)
        if result.returncode == 0:
            print("   ✅ ffmpeg found")
        else:
            print("   ❌ ffmpeg not working properly")
            return False
    except FileNotFoundError:
        print("   ❌ ffmpeg not found - please install ffmpeg")
        print("      Download from: https://ffmpeg.org/download.html")
        return False
    
    return True

# === MAIN EXECUTION ===
if __name__ == "__main__":
    print("🎵 YouTube Audio Downloader for Spotify Playlist")
    print("=" * 50)
    print("⚠️  IMPORTANT LEGAL NOTICE:")
    print("   This tool is for educational purposes only.")
    print("   Only download content you have the right to access.")
    print("   Respect copyright laws and platform terms of service.")
    print("   Consider using official music streaming services.")
    print("=" * 50)
    
    if not check_prerequisites():
        print("❌ Prerequisites not met. Please install required tools.")
        exit(1)
    
    # Get input file
    tracks_file = input("\nEnter path to your extracted tracks JSON file: ").strip()
    if not tracks_file:
        # Look for recent extraction files
        import glob
        json_files = glob.glob("*_data.json")
        if json_files:
            tracks_file = json_files[-1]
            print(f"Using: {tracks_file}")
        else:
            print("❌ No tracks file found")
            exit(1)
    
    # Optional parameters
    start_from = input("Start from track number (default: 1): ").strip()
    start_from = int(start_from) if start_from.isdigit() else 1
    
    max_downloads = input("Maximum downloads (default: all): ").strip()
    max_downloads = int(max_downloads) if max_downloads.isdigit() else None
    
    output_folder = input("Output folder (default: auto-generated): ").strip()
    output_folder = output_folder if output_folder else None
    
    # Start download
    download_playlist_audio(tracks_file, output_folder, start_from, max_downloads)

🎵 YouTube Audio Downloader for Spotify Playlist
⚠️  IMPORTANT LEGAL NOTICE:
   This tool is for educational purposes only.
   Only download content you have the right to access.
   Respect copyright laws and platform terms of service.
   Consider using official music streaming services.
🔧 Checking prerequisites...
   ❌ ffmpeg not found - please install ffmpeg
      Download from: https://ffmpeg.org/download.html
❌ Prerequisites not met. Please install required tools.
🎵 Starting download of 276 tracks...
📁 Output folder: downloaded_playlist_20250722_122243
⚠️  LEGAL NOTICE: Only download content you have the right to access.
⚠️  Be aware of copyright laws in your jurisdiction.

🎵 [1/276] Nee Kavithaigala - Dhibu Ninan Thomas, Pradeep Kumar


ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location


   ⚠️  Attempt 1 failed: ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location, retrying...


ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location


   ⚠️  Attempt 2 failed: ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location, retrying...


...

   ⚠️  Attempt 1 failed: ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location, retrying...


ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location


   ⚠️  Attempt 2 failed: ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location, retrying...


ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location


   ❌ Failed: ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location

📊 DOWNLOAD COMPLETE
   Total tracks: 276
   ✅ Successful: 0
   ❌ Failed: 276
   📈 Success rate: 0.0%
   📁 Files saved in: downloaded_playlist_20250722_122243
   📋 Detailed log: downloaded_playlist_20250722_122243\download_log.txt
   📊 Summary: downloaded_playlist_20250722_122243\download_summary.json
